In [1]:
#Installations in case not there
! pip install transformers datasets evaluate
! pip uninstall -y transformers accelerate
! pip install transformers accelerate
! pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.9 MB/s eta 0:00:00
Found existing installation: transformers 4.28.1
Uninstalling transformers-4.28.1:
  Successfully uninstalled transformers-4.28.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 14.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.13.4
    Uninstalling huggingface-hub-0.13.4:
      Successfully uninstalled huggingface-hub-0.13.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 4.6 MB/s eta 0:00:00a 0:00:01


In [2]:
#Download BERT base
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/opt/anaconda3/envs/mlclass/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import pandas as pd
def load_data(csv):
  # Reads the raw csv file and split into
  # sentences (x) and target (y)
  df = pd.read_csv(csv)  
  return df
data = load_data(f'{PATH}/data.csv')
data=data.rename(columns = {'type':'labels'})
print(data)

        Unnamed: 0                                                url   
0                0  https://en.wikipedia.org/wiki/Sexhow%20railway...  \
1                1        https://en.wikipedia.org/wiki/Eti%C3%A4inen   
2                2  https://en.wikipedia.org/wiki/Inverse%20functi...   
3                3  https://en.wikipedia.org/wiki/Stepping%20on%20...   
4                4        https://en.wikipedia.org/wiki/Rob%20Bradley   
...            ...                                                ...   
299995      149995        https://en.wikipedia.org/wiki/Randy%20Borum   
299996      149996  https://en.wikipedia.org/wiki/Sa%27och%20language   
299997      149997    https://en.wikipedia.org/wiki/Philip%20Hanawalt   
299998      149998  https://en.wikipedia.org/wiki/Vossius%20Gymnasium   
299999      149999    https://en.wikipedia.org/wiki/Simone%20Stratigo   

                           title   
0         Sexhow railway station  \
1                       Etiäinen   
2       Inverse

In [ ]:
#Tokenize inputs
def preprocess_function(examples):
    return tokenizer(examples["intro"], truncation=True)

In [ ]:
#Create huggingface dataset style object
from datasets import Dataset
import numpy as np
train, validate, test = \
              np.split(data.sample(frac=1, random_state=42), 
                       [int(.6*len(data)), int(.8*len(data))])
train = Dataset.from_pandas(train)
val = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)
tokenized_df_train = train.map(preprocess_function, batched=True)
tokenized_df_val = val.map(preprocess_function, batched=True)
tokenized_df_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/180000 [00:00<?, ? examples/s]

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [ ]:
#Create data collator for feeding into model
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from datasets import load_metric

accuracy_metric = load_metric('accuracy')
precision_metric = load_metric('precision')
recall_metric = load_metric('recall')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels)
    recall = recall_metric.compute(predictions=predictions, references=labels)
    
    return {
        'accuracy': accuracy["accuracy"],
        'precision': precision["precision"],
        'recall': recall["recall"],
        'Type I Error': 1 - precision["precision"],  # Type I error is equivalent to (1 - Precision)
        'Type II Error': 1 - recall["recall"],  # Type II error is equivalent to (1 - Recall)
    }


In [ ]:
#Establish label dictionaries 
id2label = {0: "HUMAN", 1: "MACHINE"}
label2id = {"HUMAN": 0, "MACHINE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)
training_args = TrainingArguments(
    output_dir="output",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
   # label_names = "labels",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_df_train,
    eval_dataset=tokenized_df_val,  # use the validation dataset for evaluation
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


trainer.train()




NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
# get the training and validation loss and accuracy values from the trainer object
history = trainer.state.log_history
train_losses = [entry['loss'] for entry in history if 'loss' in entry]
eval_losses = [entry['eval_loss'] for entry in history if 'eval_loss' in entry]


# Plot training and validation loss over time
plt.figure(figsize=(15,8))
plt.title("Training and Validation Loss")
plt.plot(train_losses,label="training_loss")
plt.plot(eval_losses,label="validation_loss")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.legend()
plt.show()


NameError: ignored

In [ ]:
train_accuracies = [entry['accuracy'] for entry in history if 'accuracy' in entry]
eval_accuracies = [entry['eval_accuracy'] for entry in history if 'eval_accuracy' in entry]

# Plot training and validation accuracy over time
plt.figure(figsize=(15,8))
plt.title("Training and Validation Accuracy")
plt.plot(train_accuracies, label="training_accuracy")
plt.plot(eval_accuracies, label="validation_accuracy")
plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Make predictions on validation set
predictions = trainer.predict(tokenized_df_val)
# The Trainer hides the columns that are not used by the model 
# (here `labels` because it was not used when creating the dataset), 
# so we set them again
predictions.label_ids = tokenized_df_val['labels']

# Compute the confusion matrix
cm = confusion_matrix(predictions.label_ids, np.argmax(predictions.predictions, axis=-1))

# Plot the confusion matrix
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()


In [ ]:
# Evaluate the model on the test set
test_results = trainer.evaluate(tokenized_df_test)

print(f'Final performance on test set: {test_results}')